In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from utils import *
import torchvision.datasets as datasets
import time
import vgg_test as vgg_test

In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [4]:
writer = SummaryWriter(flush_secs=10)
model = vgg_test.VGG(vgg_test.make_layers(vgg_test.cfg['B'], batch_norm=True, writer=writer), writer=writer)

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, 4),
        transforms.ToTensor(),
        normalize,
    ]), download=True),
    batch_size=128, shuffle=True,
    num_workers=5, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=128, shuffle=False,
    num_workers=5, pin_memory=True)

# define loss function (criterion) and pptimizer
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified


In [6]:
optimizer = torch.optim.Adam(model.parameters(), 3e-4,
                                weight_decay=0.0005)

In [7]:
def train(train_loader, model, criterion, optimizer, epoch,writer = SummaryWriter(flush_secs=10)):
    """
        Run one train epoch
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))
        writer.add_scalar('Loss/Train', losses.val)
        writer.add_scalar('Prec@1', top1.val)

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
def validate(val_loader, model, criterion):
    """
    Run evaluation
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        with torch.no_grad():
            output = model(input)
            loss = criterion(output, target)

        output = output.float()
        loss = loss.float()

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 200 == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(val_loader), batch_time=batch_time, loss=losses,
                      top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))

    return top1.avg

In [8]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [9]:
train_epochs = 20
for i in range(train_epochs):
    train(train_loader, model, criterion, optimizer, i, writer=writer)


Epoch: [0][0/391]	Time 2.879 (2.879)	Data 0.131 (0.131)	Loss 2.3018 (2.3018)	Prec@1 10.156 (10.156)
Epoch: [0][100/391]	Time 2.712 (2.646)	Data 0.001 (0.002)	Loss 1.8017 (2.1109)	Prec@1 29.688 (17.760)
Epoch: [0][200/391]	Time 2.532 (2.660)	Data 0.001 (0.001)	Loss 1.7289 (1.9608)	Prec@1 39.062 (24.036)
Epoch: [0][300/391]	Time 2.547 (2.627)	Data 0.001 (0.001)	Loss 1.6663 (1.8540)	Prec@1 38.281 (28.473)
Epoch: [1][0/391]	Time 2.828 (2.828)	Data 0.066 (0.066)	Loss 1.5148 (1.5148)	Prec@1 42.969 (42.969)
Epoch: [1][100/391]	Time 2.540 (2.574)	Data 0.001 (0.001)	Loss 1.4913 (1.4580)	Prec@1 41.406 (45.498)
Epoch: [1][200/391]	Time 2.588 (2.576)	Data 0.001 (0.001)	Loss 1.4236 (1.4226)	Prec@1 44.531 (47.349)
Epoch: [1][300/391]	Time 2.571 (2.579)	Data 0.001 (0.001)	Loss 1.2627 (1.3835)	Prec@1 53.125 (49.131)
Epoch: [2][0/391]	Time 2.817 (2.817)	Data 0.067 (0.067)	Loss 1.3881 (1.3881)	Prec@1 46.094 (46.094)
Epoch: [2][100/391]	Time 2.602 (2.574)	Data 0.001 (0.001)	Loss 1.0374 (1.1612)	Prec@1 62

In [10]:
validate(val_loader, model, criterion)

Test: [0/79]	Time 3.003 (3.003)	Loss 0.2793 (0.2793)	Prec@1 90.625 (90.625)
 * Prec@1 86.870


86.87

In [13]:
# torch.save(model.state_dict(), 'model_scale.pt')

In [ ]:
model_wo = torchvision.models.vgg11()
optimizer_wo = torch.optim.Adam(model_wo.parameters(), 3e-4,
                                weight_decay=0.0005)

In [ ]:
train(train_loader, model_wo, criterion, optimizer_wo, 10000)

In [ ]:
print('validation_error: ' , get_model_error(model, val_loader))